# Load all fiducial points and labels

In [6]:
import scipy.io
import os
import numpy as np
import pandas as pd

dir_list = os.listdir('train_set/points')
dir_list.sort()

def get_points(file):
    mat = scipy.io.loadmat(file)
    return mat[list(mat.keys())[3]]

fiducial_pt_full = np.stack((get_points('train_set/points/'+filename) for filename in dir_list))
label_full = pd.read_csv('train_set/label.csv')['label']

### honestly don't know what this warning means... the data are loaded though

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3254: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  if (await self.run_code(code, result,  async_=asy)):


# Train-test split

In [7]:
from sklearn.model_selection import train_test_split

# note: the train and test set are both imbalanced since I didn't split by proportion of labels
X_train, X_test, y_train, y_test = train_test_split(fiducial_pt_full, label_full, test_size=0.2, random_state=42)

### Question: should be split the data so that we have a balanced test set? (which is what will be given on present day)

# Baseline model (Gradient boosting tree)

In [26]:
from sklearn.metrics import pairwise_distances

# extract pairwise distance as features (78*77/2=3303 features)
# nrow=number of training data; ncol=3303
feature_train = np.stack((pairwise_distances(X_train[i])[np.triu_indices(78, k = 1)] for i in range(X_train.shape[0])))
feature_test = np.stack((pairwise_distances(X_test[i])[np.triu_indices(78, k = 1)] for i in range(X_test.shape[0])))


/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3254: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  if (await self.run_code(code, result,  async_=asy)):
/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3254: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  if (await self.run_code(code, result,  async_=asy)):


In [22]:
from sklearn.ensemble import GradientBoostingClassifier

### there are lots of parameter to tune here
### could have used cross-validation to perform model selection (e.g. different GBT with different parameters)

# train the model (about 5 mins)
gbt = GradientBoostingClassifier(random_state=0)
gbt.fit(feature_train, y_train)

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=0, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [27]:
# mean accuracy on test data
gbt.score(feature_test, y_test)

0.8066666666666666